# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [ ]:
# Environment variables
import os
os.environ["GROQ_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.7 MB/s eta 0:00:00


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
%pip install --quiet faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 63.5 MB/s eta 0:00:00


### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [5]:
from groq import Groq
from typing import List
import nest_asyncio

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

MODEL_PROVIDER = "groq"
MODEL_NAME = "openai/gpt-oss-120b"
APP_VERSION = 1.0

RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""

groq_client = Groq()
nest_asyncio.apply()

def get_vector_db_retriever():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    docs = [
        "LangSmith helps trace and debug LLM apps.",
        "Groq provides extremely fast inference for LLaMA and Mixtral models.",
        "RAG pipelines combine retrieval with generation for better answers."
    ]

    # Build FAISS vector DB
    vectorstore = FAISS.from_texts(docs, embeddings)

    return vectorstore.as_retriever()

retriever = get_vector_db_retriever()

from langsmith import traceable

@traceable(metadata={"vectordb": "faiss", "embeddings": "huggingface"})
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    return call_groq(messages)

@traceable(metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER})
def call_groq(messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0):
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

/tmp/ipython-input-873064958.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

@traceable handles the RunTree lifecycle for you!

In [6]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Use LangSmith’s `@traceable` decorator to wrap any function you want to monitor. Import it (`from langsmith import traceable`), apply `@traceable` above the function definition, and when the function runs LangSmith automatically records inputs, outputs, and any LLM calls for tracing and debugging. Make sure your LangSmith API key is set so the traces are sent to your workspace.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [7]:
from langsmith import traceable
from typing import List

groq_client = Groq()

@traceable(
    metadata={"vectordb": "faiss", "embeddings": "huggingface"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_groq(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_groq(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [8]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Use the @traceable decorator’s metadata argument when you define the function. For example:

```python
@traceable(metadata={"user_id": uid, "source": "api"})
def my_llm_call(...):
    …
```

The provided dictionary will be attached to the LangSmith run as its metadata.


You can also add metadata at runtime!

In [9]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

In LangSmith you can attach extra data to a running trace by calling the `add_metadata` (or `update`) method on the `Run` object (or via `client.update_run(run_id, metadata=…)`) while your code is executing. Pass a dictionary of key‑value pairs you want to record, and LangSmith will merge it into the run’s metadata. This lets you enrich the trace at runtime without restarting the run.


##### Let's take a look in LangSmith!

## **MY EXAMPLES**

In [14]:
question = "What are the benefits of using LangSmith for debugging RAG applications?"
ai_answer = langsmith_rag(question)
print(ai_answer)

LangSmith lets you trace each step of a RAG pipeline, so you can see exactly how retrieval and generation interact. This visibility makes it easy to pinpoint where errors or poor answers arise and to debug them quickly. As a result, you can iteratively improve the RAG application’s performance and reliability.


In [15]:
question = "What kind of information is captured by default when using `@traceable`?"
ai_answer = langsmith_rag(question)
print(ai_answer)

By default the `@traceable` decorator records the function’s input arguments, its return value, and any LLM calls made inside (including prompts, responses and token usage). It also logs metadata such as the function name, timestamps, and execution duration. This information lets LangSmith trace and debug the end‑to‑end flow of your LLM app.


In [16]:
question = "How can I update metadata for a run after it has finished?"
ai_answer = langsmith_rag(question)
print(ai_answer)

You can modify a finished run’s metadata through LangSmith’s API (or the UI). Call the `update_run` endpoint (or use the SDK’s `client.update_run(run_id, metadata={…})`) with the run’s ID and the new metadata fields you want to set. The changes are applied immediately even after the run has completed.
